In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_probability as tfp
import gensim
import random
import matplotlib.pyplot as plt
from typing import Union
from mlp import *
from text_data_utils import *
from bvae import *

TensorFlow 2.x selected.


In [0]:
max_len = 120
train_summaries, train_codes = load_iyer_file("train.txt", max_len=max_len)
summaries_wv = gensim.models.Word2Vec(train_summaries, size=100, min_count=5).wv
codes_wv = gensim.models.Word2Vec(train_codes, size=100, min_count=5).wv
train_summaries_tensor = tokenized_texts_to_tensor(train_summaries, summaries_wv, max_len)
train_codes_tensor = tokenized_texts_to_tensor(train_codes, codes_wv, max_len)
train_summaries_tensor_fl = np.reshape(train_summaries_tensor,
                                        (train_summaries_tensor.shape[0],
                                        train_summaries_tensor.shape[1] * train_summaries_tensor.shape[2]))
train_codes_tensor_fl = np.reshape(train_codes_tensor,
                                    (train_codes_tensor.shape[0],
                                    train_codes_tensor.shape[1] * train_codes_tensor.shape[2]))

val_summaries, val_codes = load_iyer_file("valid.txt", max_len=max_len)
val_summaries_tensor = tokenized_texts_to_tensor(val_summaries, summaries_wv, max_len)
val_codes_tensor = tokenized_texts_to_tensor(val_codes, codes_wv, max_len)
val_summaries_tensor_fl = np.reshape(val_summaries_tensor,
                                      (val_summaries_tensor.shape[0],
                                      val_summaries_tensor.shape[1] * val_summaries_tensor.shape[2]))
val_codes_tensor_fl = np.reshape(val_codes_tensor,
                                  (val_codes_tensor.shape[0],
                                  val_codes_tensor.shape[1] * val_codes_tensor.shape[2]))

test_summaries, test_codes = load_iyer_file("test.txt", max_len=max_len)
test_summaries_tensor = tokenized_texts_to_tensor(test_summaries, summaries_wv, max_len)
test_codes_tensor = tokenized_texts_to_tensor(test_codes, codes_wv, max_len)
test_summaries_tensor_fl = np.reshape(test_summaries_tensor,
                                      (test_summaries_tensor.shape[0],
                                        test_summaries_tensor.shape[1] * test_summaries_tensor.shape[2]))
test_codes_tensor_fl = np.reshape(test_codes_tensor,
                                  (test_codes_tensor.shape[0],
                                    test_codes_tensor.shape[1] * test_codes_tensor.shape[2]))

latent_dim = 512

In [8]:
model = BimodalVariationalAutoEncoder(train_summaries_tensor_fl.shape[1],
                                      train_codes_tensor_fl.shape[1],
                                      latent_dim)

model.train(train_summaries_tensor_fl, train_codes_tensor_fl, val_summaries_tensor_fl, val_codes_tensor_fl, 35, 128,
            tf.keras.optimizers.Adam(learning_rate=0.0001))

Epoch 1 of 35 completed, training loss = 1.5704983472824097, validation loss = 0.7900749444961548
Epoch 2 of 35 completed, training loss = 0.7295081615447998, validation loss = 0.6647806763648987
Epoch 3 of 35 completed, training loss = 0.6380577087402344, validation loss = 0.5988606810569763
Epoch 4 of 35 completed, training loss = 0.588570237159729, validation loss = 0.5638865232467651
Epoch 5 of 35 completed, training loss = 0.5585728883743286, validation loss = 0.5348657965660095
Epoch 6 of 35 completed, training loss = 0.5315206050872803, validation loss = 0.512114405632019
Epoch 7 of 35 completed, training loss = 0.5106552839279175, validation loss = 0.49523302912712097
Epoch 8 of 35 completed, training loss = 0.4961041808128357, validation loss = 0.48308151960372925
Epoch 9 of 35 completed, training loss = 0.4835800230503082, validation loss = 0.4695216417312622
Epoch 10 of 35 completed, training loss = 0.4681086540222168, validation loss = 0.4544864296913147
Epoch 11 of 35 comp

### Example from train set

In [0]:
random.seed()
random_idx = random.randrange(train_summaries_tensor.shape[0])
random_train_summary = np.array([train_summaries_tensor[random_idx]])
random_train_code = np.array([train_codes_tensor[random_idx]])

In [10]:
tensor_to_tokenized_texts(random_train_summary, summaries_wv)[0]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['<s>', 'linq', 'to', 'xml', 'extract', 'nested', 'elements', '</s>']

In [11]:
tensor_to_tokenized_texts(random_train_code, codes_wv)[0]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['<s>',
 'var',
 'result',
 '=',
 'from',
 'p',
 'in',
 'xmlDoc',
 '.',
 'Descendants',
 '(',
 '"',
 'person',
 '"',
 ')',
 '\\',
 'n',
 'from',
 'a',
 'in',
 'p',
 '.',
 'Descendants',
 '(',
 '"',
 'address',
 '"',
 ')',
 '\\',
 'n',
 'where',
 'p',
 '.',
 'Element',
 '(',
 '"',
 '<UNK>',
 '"',
 ')',
 '.',
 'Value',
 '=',
 '=',
 '"',
 '2',
 '"',
 '\\',
 'n',
 'select',
 'new',
 '\\',
 'n',
 '{',
 '\\',
 'n',
 'City',
 '=',
 'a',
 '.',
 'Element',
 '(',
 '"',
 'city',
 '"',
 ')',
 '.',
 'Value',
 ',',
 '\\',
 'n',
 'Location',
 '=',
 'a',
 '.',
 'Element',
 '(',
 '"',
 'location',
 '"',
 ')',
 '.',
 'Value',
 '\\',
 'n',
 '}',
 ';',
 '</s>']

In [24]:
tensor_to_tokenized_texts(np.reshape(model.language_decoder(model.language_encoder(np.reshape(random_train_summary, (1, random_train_summary.shape[1] * random_train_summary.shape[2]))).mean()), (1, train_summaries_tensor.shape[1], train_summaries_tensor.shape[2])), summaries_wv)[0]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['<s>', 'concatenate', 'grouped', 'seperated', 'seperated', '</s>']

In [25]:
tensor_to_tokenized_texts(np.reshape(model.language_decoder(model.source_code_encoder(np.reshape(random_train_code, (1, random_train_code.shape[1] * random_train_code.shape[2]))).mean()), (1, train_summaries_tensor.shape[1], train_summaries_tensor.shape[2])), summaries_wv)[0]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['<s>', 'groupby', 'groupby', 'seperated', 'procedures', '</s>']

In [26]:
tensor_to_tokenized_texts(np.reshape(model.source_code_decoder(model.source_code_encoder(np.reshape(random_train_code, (1, random_train_code.shape[1] * random_train_code.shape[2]))).mean()), (1, train_codes_tensor.shape[1], train_codes_tensor.shape[2])), codes_wv)[0]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['<s>',
 'var',
 'query',
 '=',
 'from',
 'tag',
 'in',
 'ctx',
 '.',
 '<UNK>',
 'join',
 '<UNK>',
 'in',
 '<UNK>',
 'in',
 'db',
 'equals',
 'equals',
 'equals',
 'equals',
 'equals',
 'ID',
 'equals',
 'Id',
 'equals',
 'ID',
 'q',
 'Id',
 '.',
 'ID',
 'select',
 'ID',
 'Id',
 'FirstName',
 'Id',
 'Id',
 '=',
 'grp',
 'equals',
 'dc',
 'select',
 'select',
 'grp',
 'u',
 '<UNK>',
 'Element',
 '<UNK>',
 '\\',
 '<UNK>',
 '<UNK>',
 'Index',
 '<UNK>',
 'Id',
 'LastName',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 'Element',
 '<UNK>',
 'State',
 '<UNK>',
 'name',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 'xyz',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 'new',
 'Index',
 ']',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 'IServiceProvider',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 'PropertyGrid',
 '(',
 'private',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 'IServiceProvider',
 '<UNK>',
 '<UNK>',
 '<UNK>',
 'class',
 'thru',
 'new',
 'PropertyGrid',
 '<UNK>',
 'build',
 'PropertyGrid',
 '<UN

In [29]:
tensor_to_tokenized_texts(np.reshape(model.source_code_decoder(model.language_encoder(np.reshape(random_train_summary, (1, random_train_summary.shape[1] * random_train_summary.shape[2]))).mean()), (1, train_codes_tensor.shape[1], train_codes_tensor.shape[2])), codes_wv)[0]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['<s>', 'var', 'q', '=', '</s>']

In [33]:
codes_wv.similar_by_vector(np.reshape(model.source_code_decoder(model.language_encoder(np.reshape(random_train_summary, (1, random_train_summary.shape[1] * random_train_summary.shape[2]))).mean()), (1, train_codes_tensor.shape[1], train_codes_tensor.shape[2]))[0,4])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('</s>', 0.9070366621017456),
 ('clip', 0.5404481887817383),
 ('someValue', 0.5296422243118286),
 ('FontSize', 0.5096644759178162),
 ('stock', 0.5088307857513428),
 ('RaisePropertyChanged', 0.5086286664009094),
 ('Utility', 0.49486249685287476),
 ('NotifyPropertyChanged', 0.4900093972682953),
 ('CD', 0.48767897486686707),
 ('nreturn', 0.4856686592102051)]